## 160: Recuperación de contraseña con token temporal (FastAPI + JWT)

Crear un sistema para permitir a los usuarios restablecer su contraseña a través de un enlace con token temporal, simulando un sistema de recuperación vía correo.

🧩 Estructura del flujo
Usuario solicita restablecer su contraseña con su correo.

El sistema genera un token JWT temporal y lo "envía" (simulado).

El usuario usa ese token para establecer una nueva contraseña.

🧩 Código completo
python
Copiar
Editar
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel, EmailStr
from sqlalchemy.orm import Session
from datetime import datetime, timedelta
from jose import jwt, JWTError
from passlib.context import CryptContext
from fastapi.security import OAuth2PasswordBearer

# --- Reutiliza base de datos y modelos de días anteriores ---
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

app = FastAPI()
DATABASE_URL = "sqlite:///./usuarios.db"
SECRET_KEY = "CLAVE_ULTRA_SECRETA"
ALGORITHM = "HS256"
RESET_TOKEN_EXPIRE_MINUTES = 15

Base = declarative_base()
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(bind=engine, autoflush=False)
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

class Usuario(Base):
    __tablename__ = "usuarios"
    id = Column(Integer, primary_key=True, index=True)
    email = Column(String, unique=True, index=True)
    contraseña = Column(String)
    rol = Column(String)

Base.metadata.create_all(bind=engine)

# --- Modelos Pydantic ---
class SolicitudReset(BaseModel):
    email: EmailStr

class NuevaContraseña(BaseModel):
    token: str
    nueva_contraseña: str

# --- Funciones utilitarias ---
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

def hashear_contraseña(contraseña: str) -> str:
    return pwd_context.hash(contraseña)

# --- 1. Solicitar recuperación de contraseña ---
@app.post("/reset-password/solicitar")
def solicitar_reset(data: SolicitudReset, db: Session = Depends(get_db)):
    usuario = db.query(Usuario).filter(Usuario.email == data.email).first()
    if not usuario:
        raise HTTPException(status_code=404, detail="Usuario no encontrado")

    expiration = datetime.utcnow() + timedelta(minutes=RESET_TOKEN_EXPIRE_MINUTES)
    token_data = {"sub": usuario.email, "exp": expiration}
    token = jwt.encode(token_data, SECRET_KEY, algorithm=ALGORITHM)

    # Simula el envío del enlace por correo
    return {
        "mensaje": "Se ha enviado un enlace para restablecer tu contraseña.",
        "enlace": f"http://localhost:8000/reset-password/confirmar?token={token}"
    }

# --- 2. Confirmar nueva contraseña ---
@app.post("/reset-password/confirmar")
def confirmar_reset(data: NuevaContraseña, db: Session = Depends(get_db)):
    try:
        payload = jwt.decode(data.token, SECRET_KEY, algorithms=[ALGORITHM])
        email = payload.get("sub")
        if not email:
            raise HTTPException(status_code=400, detail="Token inválido")
    except JWTError:
        raise HTTPException(status_code=400, detail="Token inválido o expirado")

    usuario = db.query(Usuario).filter(Usuario.email == email).first()
    if not usuario:
        raise HTTPException(status_code=404, detail="Usuario no encontrado")

    usuario.contraseña = hashear_contraseña(data.nueva_contraseña)
    db.commit()
    return {"mensaje": "Contraseña actualizada correctamente"}